In [111]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import folium
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings(action='ignore')

In [112]:
address=pd.DataFrame()
for i in range(1,10):
    url = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/stores/json?page=" + str(i) + '&perPage=5000'
    response = requests.get(url)
    output = json.loads(response.content)
    output_df=json_normalize(output['storeInfos'])
    
    address=pd.concat([address,output_df])

address=address.reset_index(drop=True)
address

,addr,code,lat,lng,name,type
0,충청남도 홍성군 갈산면 상촌로 12-1,34840559,36.602986,126.548911,신신약국,01
1,대구광역시 달서구 달구벌대로 1736 1층 (두류동),37844571,35.856678,128.555278,밝은온누리약국,01
2,충청남도 보령시 남대천로 58 (대천동),34833323,36.348742,126.593534,지애약국,01
3,경상북도 고령군 쌍림면 대가야로 608,5038300744,35.679196,128.242253,고령쌍림우체국,02
4,부산광역시 금정구 서동로 168 송월빌딩 1층 102호 (서동),21839824,35.214448,129.104775,굿모닝약국,01
...,...,...,...,...,...,...
26184,대구광역시 서구 달구벌대로 1805 (내당동),37834347,35.859820,128.562193,윤약국,01
26185,전라남도 여수시 신월로 712-1 (봉산동),36808415,34.734001,127.719918,남산약국,01
26186,경상북도 경산시 경안로38길 17 (중방동),37847082,35.825802,128.737720,경산제일약국,01
26187,부산광역시 동구 초량상로 93 1층 (초량동),21829420,35.120073,129.038783,백광약국,01


In [113]:
sales=pd.DataFrame()
output_df=pd.DataFrame()
for i in range(1,10):
    url = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/sales/json?page=" + str(i) + '&perPage=5000'
    response = requests.get(url)
    output = json.loads(response.content)
    output_df=json_normalize(output['sales'])
    sales=pd.concat([sales,output_df])
    
sales=sales.reset_index(drop=True).drop(sales.columns[[4,5,6]],axis='columns')
sales

,code,created_at,remain_stat,stock_at
0,31837492,2020/04/01 14:20:00,plenty,2020/04/01 12:52:00
1,21836841,2020/04/01 14:20:00,plenty,2020/04/01 09:48:00
2,37839128,2020/04/01 14:20:00,plenty,2020/03/31 17:38:00
3,11831049,2020/04/01 14:20:00,plenty,2020/03/31 12:53:00
4,21839271,2020/04/01 14:20:00,plenty,2020/04/01 09:11:00
...,...,...,...,...
25647,5148300517,2020/04/01 14:20:00,plenty,2020/04/01 09:21:00
25648,5098301396,2020/04/01 14:20:00,some,2020/03/30 09:16:00
25649,12862169,2020/04/01 14:20:00,plenty,2020/04/01 12:20:00
25650,12867527,2020/04/01 14:20:00,some,2020/04/01 08:57:00


In [114]:
final=pd.merge(address,sales,on='code')
final=final.dropna(how='any')

In [115]:
final=final.reset_index(drop=True)

In [116]:
final['remain_stat_color']=''

In [117]:
for i in range(len(final)):
    if final['remain_stat'][i]=='plenty':
        final['remain_stat'][i] = '100개 이상'
        final['remain_stat_color'][i] = 'green'
    elif final['remain_stat'][i] == 'some':
        final['remain_stat'][i] = '30개 이상 100개 미만'
        final['remain_stat_color'][i] = 'yellow'
    elif final['remain_stat'][i] =='few':
        final['remain_stat'][i] = '2개 이상 30개 미만'
        final['remain_stat_color'][i] = 'red'
    elif final['remain_stat'][i] == 'empty':
        final['remain_stat'][i] = '1개 이하'
        final['remain_stat_color'][i] = 'lightgray'
    else:
        final['remain_stat'][i] = '판매중지'
        final['remain_stat_color'][i] = 'gray'

In [118]:
final

,addr,code,lat,lng,name,type,created_at,remain_stat,stock_at,remain_stat_color
0,충청남도 홍성군 갈산면 상촌로 12-1,34840559,36.602986,126.548911,신신약국,01,2020/04/01 14:20:00,100개 이상,2020/03/30 09:57:00,green
1,대구광역시 달서구 달구벌대로 1736 1층 (두류동),37844571,35.856678,128.555278,밝은온누리약국,01,2020/04/01 14:20:00,100개 이상,2020/03/31 18:23:00,green
2,충청남도 보령시 남대천로 58 (대천동),34833323,36.348742,126.593534,지애약국,01,2020/04/01 14:20:00,100개 이상,2020/04/01 13:59:00,green
3,부산광역시 금정구 서동로 168 송월빌딩 1층 102호 (서동),21839824,35.214448,129.104775,굿모닝약국,01,2020/04/01 14:20:00,100개 이상,2020/04/01 08:44:00,green
4,경상남도 함양군 함양읍 한들로 143,38827077,35.519704,127.732118,평화약국,01,2020/04/01 14:20:00,100개 이상,2020/04/01 08:43:00,green
...,...,...,...,...,...,...,...,...,...,...
24157,대구광역시 서구 달구벌대로 1805 (내당동),37834347,35.859820,128.562193,윤약국,01,2020/04/01 14:20:00,2개 이상 30개 미만,2020/03/31 08:54:00,red
24158,전라남도 여수시 신월로 712-1 (봉산동),36808415,34.734001,127.719918,남산약국,01,2020/04/01 14:20:00,30개 이상 100개 미만,2020/04/01 09:59:00,yellow
24159,경상북도 경산시 경안로38길 17 (중방동),37847082,35.825802,128.737720,경산제일약국,01,2020/04/01 14:20:00,100개 이상,2020/04/01 08:52:00,green
24160,부산광역시 동구 초량상로 93 1층 (초량동),21829420,35.120073,129.038783,백광약국,01,2020/04/01 14:20:00,100개 이상,2020/04/01 09:48:00,green


In [119]:
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(final)):
    folium.Marker([final['lat'][i],final['lng'][i]],
                    icon=folium.Icon(color=final['remain_stat_color'][i]),
                    popup=folium.Popup('<strong>' + final['name'][i]+'</strong><br>'+
                                       '<strong>''주소 : ''</strong>' + final['addr'][i]+ '<br>'+
                                       '<strong>''재고수량 : ''</strong>'+ final['remain_stat'][i]+'<br>'+
                                       '<strong>''입고시간 : ''</strong>' + final['stock_at'][i], max_width=500)
                 ).add_to(marker_cluster)
    
map.save('corona_map.html')
